# Welcome to TQ42

## Example of training a generic time series Neural Network

In [ ]:
#Import Packages
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.dataset import list_all as list_all_datasets
from tq42.experiment_run import list_all as list_all_runs
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto

from tq42.algorithm import (
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto,
    ModelStorageInfoProto,
    ActFuncProto,
    GenericMLTrainMetadataProto,
    GenericMLTrainParametersProto,
    GenericMLInferMetadataProto,
    GenericMLInferParametersProto,
    Layer,
    PQNLayer,
    ClassicalDenseLayer,
    MLModelType,
    ActivationFunctionLayer,
    ActFuncProto,
    DropoutLayer,
    BatchNormalizationLayer,
    TrainModelInfoProto,
    MLTrainInputsProto,
    TSEvalInputsProto,
    AlgorithmProto,
    MeasurementModeProto,
    MeasureProto,
    EntanglingProto,
    DiffMethodProto,
    QubitTypeProto,
    ClassicalLSTMLayer,
    QLSTMLayer,
)

from google.protobuf.json_format import MessageToDict

# Create a client. Select an Organization, Project and Experiment

In [ ]:
# Connect to the cloud service
# https://terraquantum.io
with TQ42Client() as client:
    client.login()

In [ ]:
# list the organizations available to you 
for organization in list_all_organizations(client=client):
    print (organization)

In [ ]:
# Select an organization from the list
org_selected = list_all_organizations(client=client)[0]
print (org_selected)

In [ ]:
# List the projects within that organization
for project in list_all_projects(client=client, organization_id=org_selected.id):
    print (project)

In [ ]:
# Select a project from the list
proj_selected = list_all_projects(client=client, organization_id=org_selected.id)[0]
print(proj_selected)

In [ ]:
# List the experiments within that project
for experiment in list_all_experiments(client=client, project_id=proj_selected.id):
    print (experiment)

In [ ]:
# Select an experiment from the list
exp_selected = list_all_experiments(client=client, project_id=proj_selected.id)[0]
print(exp_selected)

In [ ]:
# List the datasets within that project
for dataset in list_all_datasets(client=client, project_id=proj_selected.id):
    print (dataset)

In [ ]:
# Select a dataset from the list
dataset_selected_train = list_all_datasets(client=client, project_id=proj_selected.id)[0]
print(dataset_selected_train)

# Build and Train the Neural Network

### Classical Example

In [ ]:
# Build the classical neural network

classical_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=30, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=45, bias=True)),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=60, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(dropout_layer=DropoutLayer(value=0.5)),
            Layer(batch_normalization_layer=BatchNormalizationLayer()),
            Layer(
                classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)
            ),  # Final layer, bias=False
        ],
        num_epochs=5,
        batch_size=128,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        k_fold=1,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_CLASSICAL_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id=dataset_selected_train.id)
    ),
),preserving_proto_field_name=True)

In [ ]:
# Train the classical neural network

train_classical = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=classical_train_message
)

print(train_classical.data)

In [ ]:
# Check the run status
print(train_classical.check())

In [ ]:
# Save the trained model ID for later use in inference
# trained_model_ID_classical = "ADD_TRAINED_MODEL_ID_HERE"
trained_model_ID_classical = "3a047329-ef52-4d73-b4f9-9747e1dac518"

### Hybrid Quantum Example

In [ ]:
# Build the hybrid quantum neural network
# Note: these example hyperparameters are chosen for speed, not model accuracy

hybrid_quantum_train_message=MessageToDict(GenericMLTrainMetadataProto(
    parameters=GenericMLTrainParametersProto(
        # Choose model type here
        model_type=MLModelType.MLP,
        # Add and customize and customize layers here
        layers=[
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=4, bias=True)),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(
                pqn_layer=PQNLayer(
                    in_features=4,
                    num_qubits=4,
                    depth=1,
                    measurement_mode=MeasurementModeProto.NONE,
                    rotation=MeasureProto.X,
                    entangling=EntanglingProto.BASIC,
                    measure=MeasureProto.Z,
                    diff_method=DiffMethodProto.ADJOINT,
                    qubit_type=QubitTypeProto.LIGHTNING_QUBIT,
                )
            ),
            Layer(
                activation_function_layer=ActivationFunctionLayer(
                    function=ActFuncProto.SIGMOID
                )
            ),
            Layer(classical_dense_layer=ClassicalDenseLayer(hidden_size=1, bias=False)),
        ],
        num_epochs=2,
        batch_size=512,
        learning_rate=0.1,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        k_fold=1,
        train_model_info=TrainModelInfoProto(
            # Provide a unique name to identify your trained model.
            name="ENTER_HYBRID_MODEL_NAME_HERE",
            # Add a brief description to help users understand the purpose or functionality of this trained model.
            description="ADD_DESCRIPTION_HERE",
        ),
    ),
    inputs=MLTrainInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id=dataset_selected_train.id)
    ),
),preserving_proto_field_name=True)

In [ ]:
# Train the hybrid quantum neural network

train_hybrid = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_TRAIN, 
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL, 
    parameters=hybrid_quantum_train_message
)

print(train_hybrid.data)

In [ ]:
# Check the run status
print(train_hybrid.check())

In [ ]:
# Save the trained model ID for later use in inference
# trained_model_ID_hybrid = "ADD_TRAINED_MODEL_ID_HERE"
trained_model_ID_hybrid = "095baa40-ce3a-48eb-a1d4-2426c154bc62"

# Use a trained model for Inference

#### Use the trained model IDs that were saved above for inference on a new dataset

In [ ]:
# List the datasets within that project
for dataset in list_all_datasets(client=client, project_id=proj_selected.id):
    print (dataset)

In [ ]:
# Select a dataset from the list
dataset_selected_infer = list_all_datasets(client=client, project_id=proj_selected.id)[1]
print(dataset_selected_infer)

### Classical Example

In [ ]:
classical_infer_message =  MessageToDict(GenericMLInferMetadataProto(
    parameters=GenericMLInferParametersProto(),
    inputs=TSEvalInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id=dataset_selected_infer.id),
        # Provide the specific storage ID of the previously trained model.      
        model=ModelStorageInfoProto(storage_id=trained_model_ID_classical),
    )
), preserving_proto_field_name=True)

In [ ]:
infer_classical = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_INFER,
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL,
    parameters=classical_infer_message,
)
print(infer_classical.data)

In [ ]:
# Check the run status
print(infer_classical.check())

### Hybrid Quantum Example

In [ ]:
hybrid_infer_message =  MessageToDict(GenericMLInferMetadataProto(
    parameters=GenericMLInferParametersProto(),
    inputs=TSEvalInputsProto(
        # Provide the specific dataset storage ID of the data you uploaded to TQ42.
        data=DatasetStorageInfoProto(storage_id=dataset_selected_infer.id),
        # Provide the specific storage ID of the previously trained model.      
        model=ModelStorageInfoProto(storage_id=trained_model_ID_classical),
    )
), preserving_proto_field_name=True)

In [ ]:
infer_hybrid = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.GENERIC_ML_INFER,
    experiment_id=exp_selected.id,
    compute=HardwareProto.SMALL,
    parameters=hybrid_infer_message,
)
print(infer_hybrid.data)

In [ ]:
# Check the run status
print(infer_hybrid.check())